In [ ]:
import pandas as pd

# import datebase
file_path = "数据作业数据集.csv"
df = pd.read_csv(file_path)
2
print (df. columns)
print(df['readingScore'].describe())
#Count: 有效的阅读分数的数量（即没有缺失值的数据）。
#Mean: 平均阅读分数。
#Standard deviation (std): 分数的标准差，表示分数的离散程度。
#Min 和 Max: 最低分数和最高分数。
#25%、50%、75% 百分位数: 这些百分位数可以帮助你了解分数的分布情况。

In [ ]:
#Clean the data by handling missing values and converting categorical variables as needed

# Check for missing values and data types
missing_values = df.isnull().sum()
data_types = df.dtypes

# Display missing values and data types
missing_values, data_types
# int64 整数型； float64 小数浮点数型； raceeth 字符串 
# 文字变量 处理方法：比如将它们转换为数值或进行分类编码（one-hot encoding）

In [ ]:
# 处理缺失值
#对于数值型列，使用均值或中位数填充缺失值。
#对于分类变量 raceeth，先处理缺失值，然后进行 one-hot 编码。
import pandas as pd

file_path = "数据作业数据集.csv"
df = pd.read_csv(file_path)
#1. 使用中位数填充数值型列缺失值
numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
df[numeric_columns] = df[numeric_columns].fillna(df[numeric_columns].median())

print(df.columns)

In [ ]:
# Fill missing values for numeric columns with their median
numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
df[numeric_columns] = df[numeric_columns].fillna(df[numeric_columns].median())

# Fill missing values in raceeth with the most frequent value (mode) 众数
df['raceeth'].fillna(df['raceeth'].mode()[0], inplace=True)

# Apply one-hot encoding to raceeth
df = pd.get_dummies(df, columns=['raceeth'], drop_first=True)

# Check for any remaining missing values after processing 检查处理后的缺失值
missing_values_after = df.isnull().sum()

# Display the results of missing values after the cleaning process 输出处理后的缺失值
missing_values_after


In [ ]:
# Overall descriptive statistics for the dataset整体数据集的描述性统计
#描述包括如均值、标准差、最小值、最大值等
overall_stats = df.describe()

# Descriptive statistics for the 'readingScore' column
reading_stats = df['readingScore'].describe()



print("Overall statistics:\n", overall_stats)
print("\nReading score statistics:\n", reading_stats)


In [ ]:
# 针对相关的学习习惯列进行描述性统计
study_habits_columns = ['read30MinsADay', 'minutesPerWeekEnglish', 'computerForSchoolwork']
study_habits_stats = df[study_habits_columns].describe()

# 输出结果
print(study_habits_stats)

In [ ]:
# 绘制 readingScore 的箱线图
plt.figure(figsize=(4, 4))
sns.boxplot(y=df['readingScore'])
plt.title('Boxplot of Reading Score')
plt.ylabel('Reading Score')
plt.show()

In [ ]:
#希望通过计算相关矩阵/多元线性回归
#以此实现：来探究每个变量与reading score的线性相关性/回归系数用来表述，每个变量对reading score影响的大小

# 计算相关矩阵
correlation_matrix = df.corr()
# 只查看与 'readingScore' 相关的列；并将相关系数从大到小排列
reading_correlation = correlation_matrix['readingScore'].sort_values(ascending=False)

# 输出与阅读分数的相关性
print(reading_correlation)



from scipy.stats import pearsonr
correlation_values = []
significant_vars = []
# 选择连续变量
numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
numeric_columns = numeric_columns.drop('readingScore')  # 排除 readingScore 本身

# 逐个计算每个连续变量与 readingScore 的相关性
#for col in numeric_columns:
    #corr, p_value = pearsonr(df[col], df['readingScore'])
    #print(f"Variable: {col}, Pearson Correlation: {corr:.2f}, P-value: {p_value:.3f}")
#逐个计算每个变量与 readingScore 的皮尔逊相关系数（corr）和显著性检验的 p 值（p_value）。
#p 值 < 0.05 表示相关性显著。
for col in numeric_columns:
    corr, p_value = pearsonr(df[col], df['readingScore'])
    # 判断 p 值是否小于 0.05
    if p_value < 0.05:
     significant_vars.append(col)# 保存显著相关的变量
     print(f"Variable: {col}, Pearson Correlation: {corr:.2f}, P-value: {p_value:.3f}")
     correlation_values.append(corr)  # 保存相关系数


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr


print(significant_vars)
print(correlation_values)

# 将 significant_vars 转换为 numpy 数组或 pandas Series，避免警告
significant_vars = np.array(significant_vars)

# 生成柱状图来展示相关性系数
plt.figure(figsize=(10, 6))
sns.barplot(x=significant_vars, y=correlation_values)
plt.title('Pearson Correlation with Reading Score for Significant Variables')
plt.xlabel('Variables')
plt.ylabel('Correlation Coefficient')
plt.xticks(rotation=45)
plt.show()


In [ ]:
#可视化相关性关系
import seaborn as sns
import matplotlib.pyplot as plt

# 计算相关矩阵
correlation_matrix = df.corr()

# 绘制相关性热图
plt.figure(figsize=(15, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix Heatmap')
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# 性别与阅读分数的箱线图
plt.figure(figsize=(4, 4))
sns.boxplot(x='male', y='readingScore', data=df)
plt.title('Gender vs Reading Score')
plt.xlabel('Gender (0 = Female, 1 = Male)')
plt.ylabel('Reading Score')
plt.show()

In [ ]:
# Hypothesis
from scipy.stats import f_oneway

#ANOVA is used to compare the means of two groups

# Grouping based on whether the mother has a bachelor's degree
bachelor_yes = df[df['motherBachelors'] == 1]['readingScore']
bachelor_no = df[df['motherBachelors'] == 0]['readingScore']

# Performing one-way ANOVA
f_stat, p_val_anova = f_oneway(bachelor_yes, bachelor_no)

print(f"F-statistic: {f_stat}, P-value: {p_val_anova}")

# Check for significance
if p_val_anova < 0.05:
    print("Mother's bachelor degree has a significant impact on reading scores.")
else:
    print("Mother's bachelor degree does not have a significant impact on reading scores.")

import seaborn as sns
import matplotlib.pyplot as plt

# 检查 'publicSchool' 列是否存在
if 'publicSchool' in df.columns:
    # 创建 'schoolType' 列
    df['schoolType'] = df['publicSchool'].apply(lambda x: 'Public School' if x == 1 else 'Private School')

    # 计算公立和私立学校学生的平均阅读分数
    mean_scores = df.groupby('schoolType')['readingScore'].mean()

    # 生成条形图
    plt.figure(figsize=(4, 4))
    sns.barplot(x=mean_scores.index, y=mean_scores.values)
    plt.title('Average Reading Scores: Public vs Private Schools')
    plt.xlabel('School Type')
    plt.ylabel('Average Reading Score')
    plt.show()
else:
    print("The 'publicSchool' column is missing from the dataset.")


In [ ]:
from scipy.stats import f_oneway

# Grouping based on whether the student attends a public school
public_school = df[df['publicSchool'] == 1]['readingScore']
private_school = df[df['publicSchool'] == 0]['readingScore']

# Performing one-way ANOVA
f_stat_public, p_val_public = f_oneway(public_school, private_school)

print(f"F-statistic: {f_stat_public}, P-value: {p_val_public}")

# Check for significance
if p_val_public < 0.05:
    print("Attending a public school has a significant impact on reading scores.")
else:
    print("Attending a public school does not have a significant impact on reading scores.")


In [ ]:
#T-statistic: This value is used to determine if the means of the two groups are statistically different from each other.
from scipy.stats import ttest_ind
# Grouping based on gender: 1 for male, 0 for female
male_scores = df[df['male'] == 1]['readingScore']
female_scores = df[df['male'] == 0]['readingScore']

# Performing t-test
t_stat, p_val = ttest_ind(male_scores, female_scores)

print(f"T-statistic: {t_stat}, P-value: {p_val}")

# Check for significance
if p_val < 0.05:
    print("Gender has a significant impact on reading scores.")
else:
    print("Gender does not have a significant impact on reading scores.")

In [ ]:
# 回归分析
import statsmodels.api as sm

# 选择特征变量（自由选择）
X = df[['male', 'preschool', 'expectBachelors', 'motherHS', 'fatherHS', 'schoolSize', 
        'read30MinsADay', 'minutesPerWeekEnglish', 'publicSchool']]

# 添加常数项
X = sm.add_constant(X)

# 因变量
y = df['readingScore']

# 拟合回归模型
model = sm.OLS(y, X).fit()

# 输出回归结果摘要
print(model.summary())

# output regression model:
   # coef: positive——predictor variable increase, dependent variable tends to increase
   # std err: standard error 标准差越小，该系数估计值越准确；标准差越大，系数估计的不确定性越大
   # t value: 系数除以标准差；|t|越大，自变量对因变量的影响越显著，|t|＞2 被认为该变量对结果有显著影响
   # p value：p值表示在原假设成立（即回归系数为0，自变量对因变量没有影响）的前提下，得到当前数据或更极端结果的概率。
        #p值小于0.05：说明该变量对因变量有显著影响，可以拒绝原假设。
        #p值大于0.05：说明该变量对因变量的影响不显著，无法拒绝原假设。
   # [0.025, 0.975] (95%置信区间)： 置信区间包含0，则变量对因变量影响不显著；不包含零，则统计学影响显著

In [ ]:


# 计算95%的置信区间
confidence_intervals = model.conf_int()

# 给置信区间结果添加列名
confidence_intervals.columns = ['Lower Bound', 'Upper Bound']

# 打印置信区间
print(confidence_intervals)

import matplotlib.pyplot as plt
import pandas as pd

# 获取回归系数及置信区间
coefficients = pd.Series(model.params, index=X.columns)
conf_intervals = model.conf_int()
conf_intervals.columns = ['Lower Bound', 'Upper Bound']

# 绘制回归系数及置信区间
plt.figure(figsize=(18, 6))
plt.errorbar(coefficients.index, coefficients, 
             yerr=[coefficients - conf_intervals['Lower Bound'], conf_intervals['Upper Bound'] - coefficients], 
             fmt='o', capsize=5)
plt.axhline(0, color='red', linestyle='--')  # 画出0的位置
plt.title('Regression Coefficients and 95% Confidence Intervals')
plt.xlabel('Variables')
plt.ylabel('Coefficient')
plt.show()

In [ ]:
#可视化回归系数

import statsmodels.api as sm
import numpy as np

# 选择特征变量
X = df[['male', 'preschool', 'expectBachelors', 'motherHS', 'fatherHS', 'schoolSize', 
        'read30MinsADay', 'minutesPerWeekEnglish', 'publicSchool']]

# 添加常数项
X = sm.add_constant(X)

# 因变量
y = df['readingScore']

# 拟合回归模型
model = sm.OLS(y, X).fit()

# 获取回归系数
coefficients = model.params

# 可视化回归系数
plt.figure(figsize=(10, 6))
coefficients[1:].plot(kind='barh', color='blue')  # 排除常数项
plt.title('Regression Coefficients')
plt.xlabel('Coefficient Value')
plt.ylabel('Features')
plt.show()

# positive coefficients： 对阅读分数有正向影响
# 条形长度： 条形越长，改变量的影响越大